In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias
# import ipynb.fs.full.featuresNLTK as f_nltk

df_train = pd.read_csv('./data/train_filtrado.csv')
# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

df_train = features.llenar_nulls(df_train)
df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)

df_clustering_train = pd.read_csv('data/clustering_train.csv')
df_clustering_test = pd.read_csv('data/clustering_test.csv')

def agregar_clustering(df, train=True):
    df_cluster = df_clustering_train if train else df_clustering_test
    df_merge = pd.merge(df, df_cluster, on='id')
    return df_merge

# df_train_f = agregar_clustering(df_train, True)
# df_test_f = agregar_clustering(df_test, False)

titulo_train = pd.read_csv('./data/titulo_ohe_train.csv')
titulo_test = pd.read_csv('./data/titulo_ohe_test.csv')
# train_DK = pd.read_csv('./data/df_train_KD.csv')
# test_DK = pd.read_csv('./data/df_test_KD.csv')

df_train_f = pd.merge(df_train, titulo_train, on= 'id', how= 'left')
df_test_f = pd.merge(df_test, titulo_test, on= 'id', how= 'left')

# df_train_f = pd.merge(df_train_f, train_DK, on= 'id', how= 'left')
# df_test_f = pd.merge(df_test_f, test_DK, on= 'id', how= 'left')

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_test_idf = pd.read_csv('./data/test_idf.csv')

df_train_f = pd.merge(df_train_f, df_train_idf, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, df_test_idf, on= 'id', how= 'left')

# df_train_f, df_test_f = utils.dividir_df_testeo(df_train_f, test_size=0.3)

In [2]:
df_test_f = features.features_independientes_precio(df_test_f)
df_test_f = features.features_dependientes_precio(df_test_f, df_train_f)

df_train_f = features.features_independientes_precio(df_train_f)
df_train_f = features.features_dependientes_precio(df_train_f, df_train_f)

# df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
# df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
# df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

# df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
# df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
# df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

df_train_f = features.rankings(df_train_f, df_train_f)
df_test_f = features.rankings(df_test_f, df_train_f)

In [3]:
df_test_f = df_test_f.drop('Unnamed: 0', axis=1, errors='ignore')
assert df_train_f.drop('precio', axis=1).columns.equals(df_test_f.drop('precio', axis=1, errors='ignore').columns)

## Selector de Features

In [4]:
from sklearn.base import BaseEstimator

class FeatureSelector(BaseEstimator):
    
    base_features = list(df_train_f.drop('precio', axis=1, errors='ignore').columns)
    
    def __init__(self, features):
        base_features = FeatureSelector.base_features

        self.features = features
        self.features_index = [i for i in range(len(base_features)) if base_features[i] in features]
        
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        return x[:, self.features_index]

## LightGBM model

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

class LightGBMWrapper(lgb.LGBMRegressor):
    
    def fit(self, x, y):
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, test_size=0.01, random_state=0)
        eval_set = [(x_eval, y_eval)]
        if self.boosting_type == 'dart':
            x_train = x
            y_train = y
            eval_set = None

        return super(LightGBMWrapper, self).fit(x_train, y_train, 
                                                early_stopping_rounds=15, eval_set=eval_set)
    
    def predict(self, X):
        return super(LightGBMWrapper, self).predict(X)

params = {
 'bagging_fraction': 0.21587742093682755,
 'bagging_freq': 0,
 'max_bin': 388,
 'max_depth': 21,
 'min_data_in_leaf': 19,
 'min_split_gain': 0.21413040034431072,
 'num_leaves': 640,
 'reg_lambda': 9.336230074572562,
#  'bagging_fraction': 0.5079900308614034,
#  'bagging_freq': 0,
#  'max_bin': 1462,
#  'max_depth': 22,
#  'min_data_in_leaf': 58,
#  'min_split_gain': 0.6819500375795696,
#  'num_leaves': 668,
#  'reg_lambda': 48.786835826989865,
 'boosting_type': 'gbdt',
 'metric': 'mae',
 'n_jobs': 4,
 'objective': 'regression',
 'n_estimators': 1500,
}

lgb_m = Pipeline(steps=[
    ('feature_selector', FeatureSelector(['antiguedad', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'piscina', 'metroscubiertos_log',
       'porcentaje_metros', 'diferencia_metros', 'metros_totales_normalizados',
       'escomercial', 'promedio_metros_tipo_propiedad',
       'promedio_metros_cub_tipo_propiedad', 'tipo_propiedad_compartida',
       'prop_frecuente', 'top_provincia', 'es_ciudad_centrica',
       'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'dolar_promedio_mes',
       'dolar_promedio_ult_mes', 'es_antigua', 'antiguedad_bins_unif',
       'antiguedad_bins_perc', 'cantidad_inquilinos', 'tam_ambientes',
       'promedio_precio_provincia', 'promedio_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'count_id_zona', 'promedio_precio_tipo_propiedad',
       'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'lat_norm',
       'lng_norm', 'promedio_precio_booleanos', 'puntaje', 'idf_titulo',
       'idf_descripcion', 'peso_titulo', 'peso_descripcion',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'distancia_ciudad_cara', 'ranking_en_provincia', 'ranking_en_ciudad',
       'ranking_en_provincia_ciudad_precio',
       'ranking_en_provincia_ciudad_cantidad',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad', 'palabra_venta',
       'palabra_casa', 'palabra_departamento', 'palabra_san',
       'palabra_polanco', 'palabra_condominio', 'palabra_lomas',
       'palabra_excelente', 'palabra_ciudad', 'palabra_méxico',
       'similares_count', 
                                          'kmeans_basicos', 'kmeans_superficie',
       'kmeans_metroscubiertos', 'kmeans_fechas', 'kmeans_latlng']
        )),
    ('lightgbm', LightGBMWrapper(**params))
])

## XGBoost

In [6]:
import xgboost as xgb

class XGBoostWrapper(xgb.XGBRegressor):
    
    def fit(self, x, y):
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, test_size=0.05, random_state=0)
        eval_set = [(x_eval, y_eval)]

        return super(xgb.XGBRegressor, self).fit(x_train, y_train, 
                                                early_stopping_rounds=10, eval_set=eval_set)
    
    def predict(self, X):
        return super(xgb.XGBRegressor, self).predict(X)

hps = {
 'verbosity': 0,
 'objective': 'reg:squarederror',
 'eval_metric': 'mae',
 'n_estimators': 1000,
 'n_jobs': 4,
 'colsample_bytree': 0.5754765853913819,
 'max_depth': 14,
 'reg_alpha': 21.057165648168983,
 'scale_pos_weight': 9,
 'subsample': 0.9968831535654209,
#  'reg_alpha': 20.91434940058063,
#  'colsample_bytree': 0.65,
 'learning_rate': 0.14,
#  'max_depth': int(16.0),
#  'scale_pos_weight': 5,
#  'subsample': 0.9569019167459879
}

xgb_m = Pipeline([
    ('feature_selector', FeatureSelector(['antiguedad', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'gimnasio', 'usosmultiples', 'piscina',
       'metrostotales_log', 'metroscubiertos_log', 'porcentaje_metros',
       'diferencia_metros', 'metros_cubiertos_normalizados', 'escomercial',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
       'tipo_propiedad_compartida', 'prop_frecuente', 'top_provincia',
       'es_ciudad_centrica', 'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'dias_desde_datos',
       'meses_desde_datos', 'dolar_promedio_mes', 'dolar_promedio_ult_mes',
       'dolar_diferencia_ult_mes', 'dolar_diferencia_mes_sum', 'delincuencia',
       'turismo', 'es_antigua', 'antiguedad_bins_unif', 'antiguedad_bins_perc',
       'cantidad_inquilinos', 'tam_ambientes', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'count_ciudad', 'promedio_id_zona',
       'count_id_zona', 'promedio_precio_tipo_propiedad',
       'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'lat_norm',
       'lng_norm', 'promedio_precio_booleanos', 'puntaje', 'idf_titulo',
       'idf_descripcion', 'peso_titulo', 'peso_descripcion',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'distancia_ciudad_cara', 'ranking_en_provincia', 'ranking_en_ciudad',
       'ranking_en_provincia_ciudad_precio',
       'ranking_en_provincia_ciudad_cantidad',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad', 
                                          'similares_count',
       'palabra_polanco', 'palabra_condominio', 'palabra_lomas',
       'palabra_excelente', 'palabra_ciudad', 'palabra_méxico',
       'palabra_terreno', 'kmeans_basicos', 'kmeans_superficie',
       'kmeans_metroscubiertos', 'kmeans_fechas', 'kmeans_latlng'])),
    ('xgboost', XGBoostWrapper(**hps))
])

## RandomForest

In [7]:
from sklearn.ensemble import RandomForestRegressor

params = {'bootstrap': False,
          'max_features': 'sqrt',
          'min_samples_split': 4,
          'n_jobs': 2,
          'n_estimators': 100}

forest_m = Pipeline([
    ('feature_selector', FeatureSelector(['antiguedad',
'count_ciudad',
'metrostotales_log',
'metroscubiertos_log',
'porcentaje_metros',
'diferencia_metros',
'metros_totales_normalizados',
'metros_cubiertos_normalizados',
'lng_norm',
'lat_norm',
'top_provincia',
'count_tipo_propiedad_ciudad',
'dia',
'dias_desde_datos',
'meses_desde_datos',
'count_tipo_propiedad',
'promedio_precio_tipo_propiedad',
'count_id_zona',
'cantidad_inquilinos',
'tam_ambientes',
'promedio_precio_provincia',
'promedio_precio_ciudad',
'idf_descripcion',
'idf_titulo',
'promedio_id_zona',
'puntaje',
'lng',
'fecha',
'metroscubiertos',
'banos',
'ranking_en_provincia_tipodepropiedad_cantidad',
'similares_count',
'ranking_en_provincia_tipodepropiedad_precio',
'lat',
'ranking_en_provincia_ciudad_cantidad',
'garages',
'ranking_en_provincia_ciudad_precio',
'ranking_en_ciudad',
'ranking_en_provincia',
'distancia_ciudad_cara',
'distancia_centro_mexico',
'distancia_ciudad_centrica',
'metrostotales'])),
    ('random_forest', RandomForestRegressor(**params))
])

## ExtraTrees

In [8]:
from sklearn.ensemble import ExtraTreesRegressor

params = {'criterion': 'mse',
 'max_features': 'sqrt',
 'min_samples_split': 4}

extratrees_m =  Pipeline([
    ('feature_selector', FeatureSelector(['antiguedad', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'idf_titulo', 'idf_descripcion',
       'peso_descripcion', 'porcentaje_metros', 'diferencia_metros',
       'metroscubiertos_bins_unif', 'metroscubiertos_bins_perc',
       'metros_totales_normalizados', 'metros_cubiertos_normalizados',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
       'promedio_metros_cubiertos_provincia', 'mes', 'dia', 'dias_desde_datos',
       'meses_desde_datos', 'antiguedad_bins_perc', 'tam_ambientes',
       'promedio_precio_provincia', 'promedio_precio_ciudad',
       'promedio_precio_ciudad_gen', 'varianza_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'promedio_id_zona_gen', 'varianza_id_zona',
       'count_id_zona', 'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'lat_norm', 'lng_norm', 'puntaje', 'distancia_ciudad_centrica',
       'distancia_centro_mexico', 'distancia_ciudad_cara',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad'])),
    ('extra_trees', ExtraTreesRegressor(n_estimators=50, **params))
])

## CatBoost

In [9]:
from catboost import CatBoostRegressor


class CatBoostWrapper(CatBoostRegressor):
    
    def fit(self, x, y):
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, test_size=0.05)
        eval_set = [(x_eval, y_eval)]

        # posiciones de features categoricas encontradas a mano
        cat_features = [0, 1, 2]
        return super(CatBoostWrapper, self).fit(x, y, cat_features=cat_features, eval_set=eval_set, early_stopping_rounds=25)


f_s = FeatureSelector(['antiguedad',
'metros_totales_normalizados',
'metros_cubiertos_normalizados',
'prop_frecuente',
'anio',
'dia',
'dias_desde_datos',
'meses_desde_datos',
'antiguedad_bins_unif',
'antiguedad_bins_perc',
'cantidad_inquilinos',
'tam_ambientes',
'promedio_precio_provincia',
'promedio_precio_ciudad',
'count_ciudad',
'promedio_id_zona',
'count_id_zona',
'promedio_precio_tipo_propiedad',
'ranking_en_provincia_tipodepropiedad_cantidad',
'ranking_en_provincia_tipodepropiedad_precio',
'ranking_en_provincia_ciudad_cantidad',
'ranking_en_provincia_ciudad_precio',
'ranking_en_ciudad',
'ranking_en_provincia',
'metroscubiertos_bins_perc',
'distancia_ciudad_cara',
'distancia_ciudad_centrica',
'puntaje',
'lng_norm',
'lat_norm',
'count_tipo_propiedad_ciudad',
'count_tipo_propiedad',
'distancia_centro_mexico',
'metroscubiertos_bins_unif',
'top_provincia',
'porcentaje_metros',
'palabra_casa',
'palabra_venta',
'centroscomercialescercanos',
'escuelascercanas',
'piscina',
'usosmultiples',
'gimnasio',
'fecha',
'lng',
'lat',
'metrostotales',
'metroscubiertos',
'banos',
'garages',
'habitaciones',
'palabra_departamento',
'diferencia_metros',
'palabra_san',
'metroscubiertos_log',
'metrostotales_log',
'similares_count',
'peso_descripcion',
'idf_titulo'] + ['provincia', 'ciudad', 'tipodepropiedad'])

params = {
    'od_type': 'Iter',
    'border_count': 176,
    'colsample_bylevel': 0.2092942139131877,
    'depth': 14,
    'l2_leaf_reg': 0.20165655817712402,
    'od_wait': 4,
    'learning_rate': 0.15,
    'iterations': 300,
    'silent': True,
    'eval_metric': 'MAE'
}


catboost_m = Pipeline([
    ('feature_selector', f_s),
    ('catboost', CatBoostWrapper(**params))
])

## Keras model

In [10]:
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from sklearn.preprocessing import MinMaxScaler

def keras_modelo():    
    model = Sequential()
    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(0.13))
    model.add(Dense(units=250, activation='relu'))
    model.add(Dropout(0.13))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.13))
    model.add(Dense(units=1, activation='linear'))

    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_squared_error'])  
    return model

keras_m = Pipeline([
    ('feature_selector', FeatureSelector(['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'clustering_label', 'idf_descripcion', 'porcentaje_metros', 'diferencia_metros',
       'promedio_metros_tipo_propiedad', 'prop_frecuente', 'top_provincia',
       'promedio_metros_totales_provincia', 'anio', 'mes',
       'trimestre', 'dias_desde_datos', 'cantidad_inquilinos', 'tam_ambientes', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'count_ciudad', 'promedio_id_zona',
       'count_id_zona','promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad', 'count_tipo_propiedad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'promedio_precio_booleanos', 'puntaje',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad'])),
    ('scaler', MinMaxScaler()),
    ('keras', KerasRegressor(build_fn=keras_modelo, epochs=5))
])

Using TensorFlow backend.
/home/javier/.thumbnails/datos/datos-tp2/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/javier/.thumbnails/datos/datos-tp2/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/javier/.thumbnails/datos/datos-tp2/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

## Stacking

In [13]:
from sklearn.preprocessing import MinMaxScaler
from vecstack import stacking
from vecstack_sk import StackingTransformer

modelos = [
#            ('keras', keras_m),
           ('catboost', catboost_m),
           ('xgboost', xgb_m),
           ('randomforest', forest_m),
           ('lightgbm', lgb_m), 
#            ('extratrees', extratrees_m),
          ]

stack = StackingTransformer(modelos, 
                          regression=True, verbose=2, n_folds=5)

stack.fit(df_train_f.drop('precio', axis=1).values, 
                          df_train_f['precio'].values)

task:         [regression]
metric:       [mean_absolute_error]
variant:      [A]
n_estimators: [4]

estimator  0: [catboost: Pipeline]
    fold  0:  [488975.01123029]
    fold  1:  [488419.92430948]
    fold  2:  [492307.16223316]
    fold  3:  [491535.91937291]
    fold  4:  [487021.22850153]
    ----
    MEAN:     [489651.84912947] + [1974.65549162]

estimator  1: [xgboost: Pipeline]
[0]	validation_0-mae:2.19737e+06
Will train until validation_0-mae hasn't improved in 10 rounds.
[1]	validation_0-mae:1.89912e+06
[2]	validation_0-mae:1.64503e+06
[3]	validation_0-mae:1.42736e+06
[4]	validation_0-mae:1.24776e+06
[5]	validation_0-mae:1.09931e+06
[6]	validation_0-mae:973247
[7]	validation_0-mae:874892
[8]	validation_0-mae:791699
[9]	validation_0-mae:726630
[10]	validation_0-mae:675822
[11]	validation_0-mae:635904
[12]	validation_0-mae:602603
[13]	validation_0-mae:577075
[14]	validation_0-mae:559338
[15]	validation_0-mae:544549
[16]	validation_0-mae:532574
[17]	validation_0-mae:522989
[18]	

[95]	validation_0-mae:490013
[96]	validation_0-mae:489946
[97]	validation_0-mae:489873
[98]	validation_0-mae:489837
[99]	validation_0-mae:489769
[100]	validation_0-mae:489698
[101]	validation_0-mae:489670
[102]	validation_0-mae:489608
[103]	validation_0-mae:489737
[104]	validation_0-mae:489727
[105]	validation_0-mae:489716
[106]	validation_0-mae:489677
[107]	validation_0-mae:489626
[108]	validation_0-mae:489623
[109]	validation_0-mae:489601
[110]	validation_0-mae:489593
[111]	validation_0-mae:489651
[112]	validation_0-mae:489524
[113]	validation_0-mae:489516
[114]	validation_0-mae:489500
[115]	validation_0-mae:489452
[116]	validation_0-mae:489423
[117]	validation_0-mae:489413
[118]	validation_0-mae:489442
[119]	validation_0-mae:489384
[120]	validation_0-mae:489373
[121]	validation_0-mae:489334
[122]	validation_0-mae:489277
[123]	validation_0-mae:489239
[124]	validation_0-mae:489259
[125]	validation_0-mae:489169
[126]	validation_0-mae:489180
[127]	validation_0-mae:489203
[128]	validatio

[50]	validation_0-mae:485152
[51]	validation_0-mae:484859
[52]	validation_0-mae:484460
[53]	validation_0-mae:484197
[54]	validation_0-mae:484069
[55]	validation_0-mae:483943
[56]	validation_0-mae:483792
[57]	validation_0-mae:483713
[58]	validation_0-mae:483748
[59]	validation_0-mae:483716
[60]	validation_0-mae:483567
[61]	validation_0-mae:483443
[62]	validation_0-mae:483357
[63]	validation_0-mae:483374
[64]	validation_0-mae:483292
[65]	validation_0-mae:483068
[66]	validation_0-mae:482999
[67]	validation_0-mae:483009
[68]	validation_0-mae:482897
[69]	validation_0-mae:482809
[70]	validation_0-mae:482810
[71]	validation_0-mae:482801
[72]	validation_0-mae:482707
[73]	validation_0-mae:482710
[74]	validation_0-mae:482612
[75]	validation_0-mae:482445
[76]	validation_0-mae:482394
[77]	validation_0-mae:482333
[78]	validation_0-mae:482221
[79]	validation_0-mae:482152
[80]	validation_0-mae:482073
[81]	validation_0-mae:482062
[82]	validation_0-mae:482042
[83]	validation_0-mae:481990
[84]	validatio

[174]	validation_0-mae:483935
[175]	validation_0-mae:483877
[176]	validation_0-mae:483836
[177]	validation_0-mae:483801
[178]	validation_0-mae:483739
[179]	validation_0-mae:483729
[180]	validation_0-mae:483712
[181]	validation_0-mae:483668
[182]	validation_0-mae:483661
[183]	validation_0-mae:483673
[184]	validation_0-mae:483630
[185]	validation_0-mae:483668
[186]	validation_0-mae:483661
[187]	validation_0-mae:483655
[188]	validation_0-mae:483646
[189]	validation_0-mae:483629
[190]	validation_0-mae:483591
[191]	validation_0-mae:483582
[192]	validation_0-mae:483548
[193]	validation_0-mae:483522
[194]	validation_0-mae:483523
[195]	validation_0-mae:483525
[196]	validation_0-mae:483502
[197]	validation_0-mae:483472
[198]	validation_0-mae:483468
[199]	validation_0-mae:483493
[200]	validation_0-mae:483485
[201]	validation_0-mae:483496
[202]	validation_0-mae:483497
[203]	validation_0-mae:483466
[204]	validation_0-mae:483401
[205]	validation_0-mae:483373
[206]	validation_0-mae:483280
[207]	vali

[102]	validation_0-mae:485202
[103]	validation_0-mae:485200
[104]	validation_0-mae:485098
[105]	validation_0-mae:485008
[106]	validation_0-mae:484922
[107]	validation_0-mae:484887
[108]	validation_0-mae:484900
[109]	validation_0-mae:484893
[110]	validation_0-mae:484749
[111]	validation_0-mae:484734
[112]	validation_0-mae:484744
[113]	validation_0-mae:484601
[114]	validation_0-mae:484578
[115]	validation_0-mae:484544
[116]	validation_0-mae:484477
[117]	validation_0-mae:484422
[118]	validation_0-mae:484416
[119]	validation_0-mae:484346
[120]	validation_0-mae:484353
[121]	validation_0-mae:484240
[122]	validation_0-mae:484082
[123]	validation_0-mae:484011
[124]	validation_0-mae:484032
[125]	validation_0-mae:483969
[126]	validation_0-mae:483981
[127]	validation_0-mae:483871
[128]	validation_0-mae:483866
[129]	validation_0-mae:483890
[130]	validation_0-mae:483945
[131]	validation_0-mae:483852
[132]	validation_0-mae:483857
[133]	validation_0-mae:483813
[134]	validation_0-mae:483811
[135]	vali

[5]	valid_0's l1: 1.07708e+06
[6]	valid_0's l1: 1.0072e+06
[7]	valid_0's l1: 944694
[8]	valid_0's l1: 891139
[9]	valid_0's l1: 843877
[10]	valid_0's l1: 801219
[11]	valid_0's l1: 763711
[12]	valid_0's l1: 731515
[13]	valid_0's l1: 702474
[14]	valid_0's l1: 677673
[15]	valid_0's l1: 655389
[16]	valid_0's l1: 637081
[17]	valid_0's l1: 621164
[18]	valid_0's l1: 606279
[19]	valid_0's l1: 593492
[20]	valid_0's l1: 581218
[21]	valid_0's l1: 571137
[22]	valid_0's l1: 562716
[23]	valid_0's l1: 555517
[24]	valid_0's l1: 548525
[25]	valid_0's l1: 542578
[26]	valid_0's l1: 537442
[27]	valid_0's l1: 532650
[28]	valid_0's l1: 528686
[29]	valid_0's l1: 525222
[30]	valid_0's l1: 521402
[31]	valid_0's l1: 518175
[32]	valid_0's l1: 516122
[33]	valid_0's l1: 513784
[34]	valid_0's l1: 511601
[35]	valid_0's l1: 509330
[36]	valid_0's l1: 507572
[37]	valid_0's l1: 506400
[38]	valid_0's l1: 505630
[39]	valid_0's l1: 504301
[40]	valid_0's l1: 502795
[41]	valid_0's l1: 501361
[42]	valid_0's l1: 500219
[43]	val

[98]	valid_0's l1: 464360
[99]	valid_0's l1: 464753
[100]	valid_0's l1: 464659
[101]	valid_0's l1: 464640
[102]	valid_0's l1: 464720
[103]	valid_0's l1: 464447
[104]	valid_0's l1: 464479
[105]	valid_0's l1: 464171
[106]	valid_0's l1: 463968
[107]	valid_0's l1: 463565
[108]	valid_0's l1: 463677
[109]	valid_0's l1: 463901
[110]	valid_0's l1: 463720
[111]	valid_0's l1: 463316
[112]	valid_0's l1: 463272
[113]	valid_0's l1: 463164
[114]	valid_0's l1: 463095
[115]	valid_0's l1: 463253
[116]	valid_0's l1: 462908
[117]	valid_0's l1: 463030
[118]	valid_0's l1: 463111
[119]	valid_0's l1: 463240
[120]	valid_0's l1: 463175
[121]	valid_0's l1: 463283
[122]	valid_0's l1: 463149
[123]	valid_0's l1: 462900
[124]	valid_0's l1: 462938
[125]	valid_0's l1: 462825
[126]	valid_0's l1: 462820
[127]	valid_0's l1: 462674
[128]	valid_0's l1: 462299
[129]	valid_0's l1: 462144
[130]	valid_0's l1: 462274
[131]	valid_0's l1: 462328
[132]	valid_0's l1: 462172
[133]	valid_0's l1: 462122
[134]	valid_0's l1: 461796
[13

[43]	valid_0's l1: 509252
[44]	valid_0's l1: 508521
[45]	valid_0's l1: 507440
[46]	valid_0's l1: 506161
[47]	valid_0's l1: 505649
[48]	valid_0's l1: 505478
[49]	valid_0's l1: 504683
[50]	valid_0's l1: 503854
[51]	valid_0's l1: 503216
[52]	valid_0's l1: 501869
[53]	valid_0's l1: 500957
[54]	valid_0's l1: 500625
[55]	valid_0's l1: 500089
[56]	valid_0's l1: 499910
[57]	valid_0's l1: 499418
[58]	valid_0's l1: 499463
[59]	valid_0's l1: 499497
[60]	valid_0's l1: 498867
[61]	valid_0's l1: 498944
[62]	valid_0's l1: 498827
[63]	valid_0's l1: 497821
[64]	valid_0's l1: 497696
[65]	valid_0's l1: 496621
[66]	valid_0's l1: 496335
[67]	valid_0's l1: 495975
[68]	valid_0's l1: 494785
[69]	valid_0's l1: 494485
[70]	valid_0's l1: 494242
[71]	valid_0's l1: 493738
[72]	valid_0's l1: 493245
[73]	valid_0's l1: 492731
[74]	valid_0's l1: 492349
[75]	valid_0's l1: 492005
[76]	valid_0's l1: 491344
[77]	valid_0's l1: 491283
[78]	valid_0's l1: 490559
[79]	valid_0's l1: 490011
[80]	valid_0's l1: 489654
[81]	valid_0

StackingTransformer(estimators=[('catboost',
                                 Pipeline(memory=None,
                                          steps=[('feature_selector',
                                                  FeatureSelector(features=['antiguedad',
                                                                            'metros_totales_normalizados',
                                                                            'metros_cubiertos_normalizados',
                                                                            'prop_frecuente',
                                                                            'anio',
                                                                            'dia',
                                                                            'dias_desde_datos',
                                                                            'meses_desde_datos',
                                                                      

In [14]:
s_train = stack.transform(df_train_f.drop('precio', axis=1).values)
s_test = stack.transform(df_test_f.drop('precio', axis=1, errors='ignore').values)

Train set was detected.
Transforming...

estimator  0: [catboost: Pipeline]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    ----
    DONE

estimator  1: [xgboost: Pipeline]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    ----
    DONE

estimator  2: [randomforest: Pipeline]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    ----
    DONE

estimator  3: [lightgbm: Pipeline]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    ----
    DONE

Transforming...

estimator  0: [catboost: Pipeline]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  

## Prediccion con todos los features + stacking

In [15]:
df_train_s = df_train_f.copy()
df_test_s = df_test_f.copy()

df_train_s['stack01'], df_train_s['stack02'], df_train_s['stack03'], df_train_s['stack04'] = zip(*s_train)
df_test_s['stack01'], df_test_s['stack02'], df_test_s['stack03'], df_test_s['stack04'] = zip(*s_test)

features_stacking = ['stack01', 'stack02', 'stack03', 'stack04']

In [16]:
df_train_s['id'] = df_train['id']
df_test_s['id'] = df_test['id']

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
from hyperopt.pyll.base import scope
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias


from sklearn.model_selection import cross_val_score
from hyperopt import STATUS_OK


features = ['antiguedad', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'piscina', 'metroscubiertos_log',
       'porcentaje_metros', 'diferencia_metros', 'metros_totales_normalizados',
       'escomercial', 'promedio_metros_tipo_propiedad',
       'promedio_metros_cub_tipo_propiedad', 'tipo_propiedad_compartida',
       'prop_frecuente', 'top_provincia', 'es_ciudad_centrica',
       'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'dolar_promedio_mes',
       'dolar_promedio_ult_mes', 'es_antigua', 'antiguedad_bins_unif',
       'antiguedad_bins_perc', 'cantidad_inquilinos', 'tam_ambientes',
       'promedio_precio_provincia', 'promedio_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'count_id_zona', 'promedio_precio_tipo_propiedad',
       'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'lat_norm',
       'lng_norm', 'promedio_precio_booleanos', 'puntaje', 'idf_titulo',
       'idf_descripcion', 'peso_titulo', 'peso_descripcion',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'distancia_ciudad_cara', 'ranking_en_provincia', 'ranking_en_ciudad',
       'ranking_en_provincia_ciudad_precio',
       'ranking_en_provincia_ciudad_cantidad',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad', 'palabra_venta',
       'palabra_casa', 'palabra_departamento', 'palabra_san',
       'palabra_polanco', 'palabra_condominio', 'palabra_lomas',
       'palabra_excelente', 'palabra_ciudad', 'palabra_méxico',
       'similares_count', 
            'kmeans_basicos', 'kmeans_superficie',
       'kmeans_metroscubiertos', 'kmeans_fechas', 'kmeans_latlng']


params_2nd = {
    
 'bagging_fraction': 0.552992696225161,
 'bagging_freq': 0,
 'max_bin': 397,
 'max_depth': 20,
 'min_data_in_leaf': 83,
 'min_split_gain': 0.9034743446375575,
 'num_leaves': 734,
 'reg_lambda': 72.68622962042346,
#  'bagging_freq': 0,
#  'max_bin': 1643,
#  'max_depth': 6,
#  'min_data_in_leaf': 62,
#  'min_split_gain': 0.8965566444331939,
#  'num_leaves': 592,
#  'reg_lambda': 35.74667217035101,
#  'bagging_freq': 0
#  'max_bin': 1012,
#  'max_depth': 5,
#  'min_data_in_leaf': 40,
#  'min_split_gain': 0.2934964048344291,
#  'num_leaves': 469,
#  'reg_lambda': 36.47905035024398,
 'boosting_type': 'dart',
 'num_boost_round': 1500,
 'objective': 'regression',
 'metric': 'mae'
}


lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(utils.filtrar_features(df_train_s, features + features_stacking).values, df_train_s['precio'].values)

In [26]:
df_test_s['target'] = lgb_m_2nd.predict(utils.filtrar_features(df_test_s, features + features_stacking))
# df_test_s = utils.pesificar_df(df_test_s, col_precio_in='target', col_precio_out='target')
df_test_s[['id', 'target']].to_csv('respuesta66.csv', index = False)

# 65: Full stacking, todos modelos, LightGBM DART de 1500
# 66: Full stacking, todos modelos, LightGBM GBDT de 1500

# print(f'MAE Stacking-full: {utils.MAE(df_test_s["precio"].values, df_test_s["target"].values)}')

In [38]:
# Sin DART:   MAE Stacking-full: 495768.2576267224

## Prediccion solo con features de stacking

In [ ]:
params_2nd = {'bagging_fraction': 0.8243831977099841,
 'bagging_freq': int(10.0),
 'feature_fraction': 0.9228324501365147,
 'learning_rate': 0.050664243951241736,
 'max_depth': int(3.0),
 'num_leaves': int(78.0),
 'objective': 'regression',
 'boosting_type': 'dart',
 'num_boost_round': 1200,
 'metric': 'mae'}


lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(s_train, df_train_f['precio'].values)

df_test_f['target'] = lgb_m_2nd.predict(s_test)
df_test_f[['id', 'target']].to_csv('respuesta47.csv', index = False)

## Prediccion con promedios

In [ ]:
df_test_f['target'] = s_test.T[1].T
df_test_f[['id', 'target']].to_csv('respuesta61.csv', index = False)

In [ ]:
from scipy.stats import mode

y_pred_test = mode(s_test, axis=1)[0]
print(f"MAE Stacking only: {utils.MAE(y_pred_test, df_test_f['precio'].values)}")

In [ ]:
import numpy as np

y_pred_test = np.average(s_test, axis=1)
print(f"MAE Stacking only: {utils.MAE(y_pred_test, df_test_f['precio'].values)}")

In [ ]:
from scipy.optimize import minimize

def mae_res(weights):
    y_pred_test = np.average(s_test, weights=weights, axis=1)
    return utils.MAE(y_pred_test, df_test_f['precio'].values)

x0 = [1] * len(s_test.T)
minimize(mae_res, x0)

In [ ]:
from scipy.optimize import minimize, differential_evolution

def mae_res(weights):
    y_pred_test = np.average(s_test, weights=weights, axis=1)
    return utils.MAE(y_pred_test, df_test_f['precio'].values)

x0 = [(-3, 4)] * len(s_test.T)
differential_evolution(mae_res, bounds=x0)

## Blending

In [15]:
from sklearn.base import clone
from sklearn.model_selection import train_test_split


class BlendingTransformer():

    def __init__(self, models, model_blend, val_split=0.10, verbose=True):
        self.models = models
        self.val_split = val_split
        self.model_blend = model_blend
        self.verbose = verbose

        
    def simple_fit(self, x, y):
        x_cat = FeatureSelector(['provincia']).transform(x)
        
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, 
                                                            test_size=self.val_split,
                                                            random_state=1,
                                                            stratify=x_cat)

        self._fit_first(x_train, y_train)
        self._fit_second(x_eval, y_eval)
        


    def fit(self, x, y, x_test):

        x_cat = FeatureSelector(['provincia']).transform(x)
        
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, 
                                                            test_size=self.val_split,
                                                            random_state=0)
#                                                             stratify=x_cat)

        self._fit_first(x_train, y_train)
        self._fit_second(x_eval, y_eval)

        self._fit_first(x, y)
        y_test = self._predict_second(x_test)

        x = np.concatenate((x, x_test))
        y = np.concatenate((y, y_test))
        x_cat = FeatureSelector(['provincia', 'intervalo_metros_cubiertos']).transform(x)
        
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, 
                                                            test_size=self.val_split, 
                                                            random_state=0)
#                                                             stratify=x_cat)
        self._fit_first(x_train, y_train)
        self._fit_second(x_eval, y_eval)

        self._fit_first(x, y)


    def _fit_first(self, x, y):

        for i in range(len(self.models)):   
            if self.verbose:
                print(f'Fitting {i}...')
            self.models[i] = clone(self.models[i])
            self.models[i].fit(x, y)


    def _fit_second(self, x, y):
        if self.verbose:
            print(f'Fitting blender...')
        self.model_blend = clone(self.model_blend)
        self.model_blend.fit(self._predict_first(x), y)


            
    def _predict_first(self, x):
        preds = []
        for i in range(len(self.models)):
            y_pred = self.models[i].predict(x)
            preds.append(y_pred)

        return np.array(preds).T


    def _predict_second(self, x):
        return self.model_blend.predict(self._predict_first(x))


    def predict(self, x):
        return self._predict_second(x)
    
    
modelos = [
           lgb_m,
           catboost_m,
           xgb_m,
           forest_m
          ]

params_2nd = {
 'bagging_freq': 0,
 'max_bin': 861,
 'max_depth': 4,
 'min_data_in_leaf': 95,
 'min_split_gain': 0.13726766148127256,
 'num_leaves': 798,
 'reg_lambda': 74.22833470483843,
 'feature_fraction': 0.85,
 'learning_rate': 0.15,
 'objective': 'regression',
 'boosting_type': 'dart',
 'num_boost_round': 1000,
 'metric': 'mae'}

# params_2nd = {'bagging_fraction': 0.8243831977099841,
#  'bagging_freq': int(10.0),
#  'feature_fraction': 0.9228324501365147,
#  'learning_rate': 0.050664243951241736,
#  'max_depth': int(3.0),
#  'num_leaves': int(78.0),
#  'objective': 'regression',
#  'boosting_type': 'dart',
#  'num_boost_round': 1200,
#  'metric': 'mae'}

lgb_m_2nd = LightGBMWrapper(**params_2nd)

blend = BlendingTransformer(modelos, lgb_m_2nd)

### Simple blend

In [ ]:
blend.simple_fit(df_train_f.drop('precio', axis=1).values, df_train_f['precio'].values)

In [ ]:
utils.MAE(blend.predict(df_test_f.drop('precio', axis=1).values), df_test_f['precio'].values)

### Long blend

In [16]:
blend.fit(df_train_f.drop('precio', axis=1).values, 
                          df_train_f['precio'].values, df_test_f.drop('precio', axis=1, errors='ignore').values)

Fitting 0...
[1]	valid_0's l1: 1.48682e+06
Training until validation scores don't improve for 15 rounds
[2]	valid_0's l1: 1.36904e+06
[3]	valid_0's l1: 1.26466e+06
[4]	valid_0's l1: 1.17256e+06
[5]	valid_0's l1: 1.09105e+06
[6]	valid_0's l1: 1.01946e+06
[7]	valid_0's l1: 956309
[8]	valid_0's l1: 900921
[9]	valid_0's l1: 853020
[10]	valid_0's l1: 810861
[11]	valid_0's l1: 772613
[12]	valid_0's l1: 738978
[13]	valid_0's l1: 710386
[14]	valid_0's l1: 684068
[15]	valid_0's l1: 662252
[16]	valid_0's l1: 642139
[17]	valid_0's l1: 624821
[18]	valid_0's l1: 610373
[19]	valid_0's l1: 597791
[20]	valid_0's l1: 585636
[21]	valid_0's l1: 574655
[22]	valid_0's l1: 566305
[23]	valid_0's l1: 557239
[24]	valid_0's l1: 549744
[25]	valid_0's l1: 543549
[26]	valid_0's l1: 537631
[27]	valid_0's l1: 532955
[28]	valid_0's l1: 528318
[29]	valid_0's l1: 524851
[30]	valid_0's l1: 521363
[31]	valid_0's l1: 518898
[32]	valid_0's l1: 515962
[33]	valid_0's l1: 513136
[34]	valid_0's l1: 509793
[35]	valid_0's l1: 50

[23]	validation_0-mae:496612
[24]	validation_0-mae:495061
[25]	validation_0-mae:493670
[26]	validation_0-mae:492391
[27]	validation_0-mae:491377
[28]	validation_0-mae:490600
[29]	validation_0-mae:489588
[30]	validation_0-mae:488516
[31]	validation_0-mae:488103
[32]	validation_0-mae:487578
[33]	validation_0-mae:487277
[34]	validation_0-mae:487002
[35]	validation_0-mae:486693
[36]	validation_0-mae:486380
[37]	validation_0-mae:486000
[38]	validation_0-mae:485519
[39]	validation_0-mae:485135
[40]	validation_0-mae:484761
[41]	validation_0-mae:484711
[42]	validation_0-mae:484702
[43]	validation_0-mae:484236
[44]	validation_0-mae:483975
[45]	validation_0-mae:483610
[46]	validation_0-mae:483267
[47]	validation_0-mae:483210
[48]	validation_0-mae:483053
[49]	validation_0-mae:483024
[50]	validation_0-mae:482902
[51]	validation_0-mae:482705
[52]	validation_0-mae:482488
[53]	validation_0-mae:482229
[54]	validation_0-mae:481977
[55]	validation_0-mae:481968
[56]	validation_0-mae:481817
[57]	validatio

/home/javier/.thumbnails/datos/datos-tp2/.venv/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/javier/.thumbnails/datos/datos-tp2/.venv/lib/python3.6/site-packages/lightgbm/callback.py:192: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


Fitting 0...
[1]	valid_0's l1: 1.46846e+06
Training until validation scores don't improve for 15 rounds
[2]	valid_0's l1: 1.35449e+06
[3]	valid_0's l1: 1.25369e+06
[4]	valid_0's l1: 1.16354e+06
[5]	valid_0's l1: 1.08461e+06
[6]	valid_0's l1: 1.01276e+06
[7]	valid_0's l1: 951018
[8]	valid_0's l1: 895692
[9]	valid_0's l1: 847325
[10]	valid_0's l1: 804256
[11]	valid_0's l1: 765525
[12]	valid_0's l1: 732263
[13]	valid_0's l1: 703683
[14]	valid_0's l1: 678763
[15]	valid_0's l1: 657782
[16]	valid_0's l1: 638999
[17]	valid_0's l1: 622655
[18]	valid_0's l1: 607295
[19]	valid_0's l1: 594772
[20]	valid_0's l1: 584672
[21]	valid_0's l1: 575547
[22]	valid_0's l1: 567059
[23]	valid_0's l1: 558917
[24]	valid_0's l1: 553262
[25]	valid_0's l1: 546901
[26]	valid_0's l1: 541649
[27]	valid_0's l1: 536820
[28]	valid_0's l1: 532865
[29]	valid_0's l1: 528871
[30]	valid_0's l1: 525732
[31]	valid_0's l1: 522506
[32]	valid_0's l1: 519737
[33]	valid_0's l1: 517761
[34]	valid_0's l1: 516171
[35]	valid_0's l1: 51

[14]	validation_0-mae:557774
[15]	validation_0-mae:542724
[16]	validation_0-mae:531120
[17]	validation_0-mae:522799
[18]	validation_0-mae:516679
[19]	validation_0-mae:511803
[20]	validation_0-mae:507395
[21]	validation_0-mae:504315
[22]	validation_0-mae:501794
[23]	validation_0-mae:499757
[24]	validation_0-mae:498121
[25]	validation_0-mae:496729
[26]	validation_0-mae:495217
[27]	validation_0-mae:494045
[28]	validation_0-mae:493230
[29]	validation_0-mae:492508
[30]	validation_0-mae:491927
[31]	validation_0-mae:491015
[32]	validation_0-mae:490359
[33]	validation_0-mae:489754
[34]	validation_0-mae:489189
[35]	validation_0-mae:488754
[36]	validation_0-mae:488436
[37]	validation_0-mae:487978
[38]	validation_0-mae:487407
[39]	validation_0-mae:486889
[40]	validation_0-mae:486843
[41]	validation_0-mae:486581
[42]	validation_0-mae:486193
[43]	validation_0-mae:486052
[44]	validation_0-mae:485750
[45]	validation_0-mae:485597
[46]	validation_0-mae:485362
[47]	validation_0-mae:485084
[48]	validatio

...

[149]	validation_0-mae:417877
[150]	validation_0-mae:417874
[151]	validation_0-mae:417850
[152]	validation_0-mae:417843
[153]	validation_0-mae:417781
[154]	validation_0-mae:417802
[155]	validation_0-mae:417809
[156]	validation_0-mae:417712
[157]	validation_0-mae:417686
[158]	validation_0-mae:417660
[159]	validation_0-mae:417674
[160]	validation_0-mae:417664
[161]	validation_0-mae:417644
[162]	validation_0-mae:417606
[163]	validation_0-mae:417576
[164]	validation_0-mae:417548
[165]	validation_0-mae:417528
[166]	validation_0-mae:417517
[167]	validation_0-mae:417513
[168]	validation_0-mae:417516
[169]	validation_0-mae:417522
[170]	validation_0-mae:417528
[171]	validation_0-mae:417514
[172]	validation_0-mae:417505
[173]	validation_0-mae:417465
[174]	validation_0-mae:417461
[175]	validation_0-mae:417467
[176]	validation_0-mae:417458
[177]	validation_0-mae:417446
[178]	validation_0-mae:417417
[179]	validation_0-mae:417444
[180]	validation_0-mae:417369
[181]	validation_0-mae:417377
[182]	vali

/home/javier/.thumbnails/datos/datos-tp2/.venv/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/javier/.thumbnails/datos/datos-tp2/.venv/lib/python3.6/site-packages/lightgbm/callback.py:192: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


Fitting 0...
[1]	valid_0's l1: 1.45142e+06
Training until validation scores don't improve for 15 rounds
[2]	valid_0's l1: 1.33264e+06
[3]	valid_0's l1: 1.22686e+06
[4]	valid_0's l1: 1.13197e+06
[5]	valid_0's l1: 1.04881e+06
[6]	valid_0's l1: 975866
[7]	valid_0's l1: 912065
[8]	valid_0's l1: 854613
[9]	valid_0's l1: 805555
[10]	valid_0's l1: 762048
[11]	valid_0's l1: 724191
[12]	valid_0's l1: 690224
[13]	valid_0's l1: 660344
[14]	valid_0's l1: 634047
[15]	valid_0's l1: 611347
[16]	valid_0's l1: 591396
[17]	valid_0's l1: 574518
[18]	valid_0's l1: 558851
[19]	valid_0's l1: 546246
[20]	valid_0's l1: 534288
[21]	valid_0's l1: 523611
[22]	valid_0's l1: 514368
[23]	valid_0's l1: 506195
[24]	valid_0's l1: 499657
[25]	valid_0's l1: 493594
[26]	valid_0's l1: 488198
[27]	valid_0's l1: 483490
[28]	valid_0's l1: 479338
[29]	valid_0's l1: 475581
[30]	valid_0's l1: 471573
[31]	valid_0's l1: 468805
[32]	valid_0's l1: 466621
[33]	valid_0's l1: 463911
[34]	valid_0's l1: 461696
[35]	valid_0's l1: 459800


[306]	valid_0's l1: 415427
[307]	valid_0's l1: 415405
[308]	valid_0's l1: 415448
[309]	valid_0's l1: 415365
[310]	valid_0's l1: 415224
[311]	valid_0's l1: 415233
[312]	valid_0's l1: 415219
[313]	valid_0's l1: 415184
[314]	valid_0's l1: 415094
[315]	valid_0's l1: 415072
[316]	valid_0's l1: 414982
[317]	valid_0's l1: 414824
[318]	valid_0's l1: 414780
[319]	valid_0's l1: 414751
[320]	valid_0's l1: 414690
[321]	valid_0's l1: 414691
[322]	valid_0's l1: 414807
[323]	valid_0's l1: 414808
[324]	valid_0's l1: 414798
[325]	valid_0's l1: 414725
[326]	valid_0's l1: 414729
[327]	valid_0's l1: 414617
[328]	valid_0's l1: 414491
[329]	valid_0's l1: 414421
[330]	valid_0's l1: 414360
[331]	valid_0's l1: 414225
[332]	valid_0's l1: 414160
[333]	valid_0's l1: 414187
[334]	valid_0's l1: 414110
[335]	valid_0's l1: 414074
[336]	valid_0's l1: 414047
[337]	valid_0's l1: 413951
[338]	valid_0's l1: 413996
[339]	valid_0's l1: 414042
[340]	valid_0's l1: 413937
[341]	valid_0's l1: 413968
[342]	valid_0's l1: 413823
[

[193]	validation_0-mae:399601
[194]	validation_0-mae:399609
[195]	validation_0-mae:399623
[196]	validation_0-mae:399624
[197]	validation_0-mae:399592
[198]	validation_0-mae:399587
[199]	validation_0-mae:399610
[200]	validation_0-mae:399569
[201]	validation_0-mae:399574
[202]	validation_0-mae:399565
[203]	validation_0-mae:399514
[204]	validation_0-mae:399466
[205]	validation_0-mae:399421
[206]	validation_0-mae:399402
[207]	validation_0-mae:399371
[208]	validation_0-mae:399356
[209]	validation_0-mae:399285
[210]	validation_0-mae:399219
[211]	validation_0-mae:399217
[212]	validation_0-mae:399195
[213]	validation_0-mae:399172
[214]	validation_0-mae:399168
[215]	validation_0-mae:399144
[216]	validation_0-mae:399156
[217]	validation_0-mae:399124
[218]	validation_0-mae:399090
[219]	validation_0-mae:399072
[220]	validation_0-mae:399069
[221]	validation_0-mae:399074
[222]	validation_0-mae:399061
[223]	validation_0-mae:399037
[224]	validation_0-mae:399047
[225]	validation_0-mae:398993
[226]	vali

In [17]:
df_test_s = df_test_f.copy()
df_test_s['target'] = blend.predict(df_test_s.drop('precio', axis=1, errors='ignore').values)
# utils.MAE(df_test_s['target'], df_test_s['precio'])
# df_test_s = utils.pesificar_df(df_test_s, col_precio_in='target', col_precio_out='target')
df_test_s[['id', 'target']].to_csv('respuesta68.csv', index = False)

# 67: LightGBM con DART

In [ ]:
df_test_f['precio']